Kaggle's San Francisco Crime Classification,  
Abdullah Alghamdi 


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, \
ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit
from matplotlib.colors import LogNorm
from sklearn.decomposition import PCA
from copy import deepcopy
%matplotlib inline  


Import the training dataset.

In [ ]:
data=pd.read_csv("/Users/hamrani/Documents/datascience/sf/train.csv")

Two functions to parse date and season to analyze the time of the incedint 

In [ ]:
# functions for time and season:

def get_time(x):
    date=datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
    time=date.hour
    day=date.day
    month=date.month
    year=date.year
    return time,day,month,year

def get_season(x):
    summer=0
    fall=0
    winter=0
    spring=0
    if (x in [5, 6, 7]):
        summer=1
    if (x in [8, 9, 10]):
        fall=1
    if (x in [11, 0, 1]):
        winter=1
    if (x in [2, 3, 4]):
        spring=1
    return summer, fall, winter, spring


    get_data function drops columns that we dont need, and add additional columns ( feature engineering )

In [ ]:
def get_data(data1,odd,oddp):
    feature=data1.columns.tolist()
    if "Descript" in feature:
        feature.remove("Descript")
    if "Resolution" in feature:
        feature.remove("Resolution")
    if "Category" in feature:
        feature.remove("Category")
    if "Id" in feature:
        feature.remove("Id")
    cData=data1[feature]
    cData.index=range(len(data1))
    address_features=cData["Address"].apply(lambda x: odd[x])
    address_features.columns=["odd"+str(x) for x in range(len(address_features.columns))]
    cData["Time"], cData["Day"], cData["Month"], cData["Year"]=zip(*cData["Dates"].apply(get_time))
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    dummy_PD = pd.get_dummies(cData['PdDistrict'], prefix='PD')
    dummy_DAY = pd.get_dummies(cData["DayOfWeek"], prefix='DAY')
    cData["oddp"]=cData["Address"].apply(lambda x: oddp[x])
    cData=cData.drop("PdDistrict",axis=1)
    cData=cData.drop("DayOfWeek",axis=1)
    cData=cData.drop("Address",axis=1)
    cData=cData.drop("Dates",axis=1)
    feature =cData.columns.tolist()
    features = cData[feature].join(dummy_PD.ix[:,:]).join(dummy_DAY.ix[:,:]).join(address_features.ix[:,:])
    features["Summer"], features["Fall"], features["Winter"], features["Spring"]=zip(*features["Month"].apply(get_season))
    if "Category" in data1.columns:
        categ= data1["Category"].astype('category')
    else:
        categ=None
    return features, categ

The most important part, where we apply the logit ( or log_loss) to the addresses column. 
https://en.wikipedia.org/wiki/Logit

In [ ]:
addresses=sorted(data["Address"].unique())
categories=sorted(data["Category"].unique())
c_num=data.groupby(["Category"]).size()
ac_num=data.groupby(["Address","Category"]).size()
a_num=data.groupby(["Address"]).size()
odd={}
oddp={}
minn=2
def_log =np.log(c_num/len(data))-np.log(1.0-c_num/float(len(data)))
for addr in addresses:
    p =a_num[addr]/float(len(data))
    oddp[addr]=np.log(p)-np.log(1.-p)
    odd[addr]=deepcopy(def_log)
    for cat in ac_num[addr].keys():
        if (ac_num[addr][cat]>minn) and ac_num[addr][cat]<a_num[addr]:
            p=ac_num[addr][cat]/float(a_num[addr])
            odd[addr][categories.index(cat)]=np.log(p)-np.log(1.0-p)
    odd[addr]=pd.Series(odd[addr])
    odd[addr].index=range(len(categories))
    
features, categ=get_data(data,odd,oddp)

Added some PCA here

In [ ]:
pca=PCA(n_components=60)
pca.fit(features)

Test it!

In [ ]:
X = features
y = categ

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

LogisticRegression works well with such a problem

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
model.score(X_train,y_train)

In [ ]:
print "logloss", log_loss(categ, model.predict_proba(features.as_matrix()))

Import the test dataset and finish the prediction

In [ ]:
test=pd.read_csv("/Users/hamrani/Documents/datascience/sf/test.csv")


In [ ]:
test_addresses = sorted(test["Address"].unique())
test_a_num = test.groupby("Address").size()
test_addr=set(test_addresses+addresses)-set(addresses)
train_addr=set(test_addresses+addresses)-set(test_addresses)
both=set(new_addresses).intersection(addresses)
for addr in test_addr:
    p = test_a_num[addr]/float(len(test)+len(data))
    oddp[addr]=np.log(p)-np.log(1.-p)
    odd[addr]=deepcopy(def_log)
    odd[addr].index=range(len(categories))
for addr in both:
    p =(a_num[addr]+test_num[addr])/float(len(test)+len(data))
    oddp[addr]=np.log(p)-np.log(1.-p)

In [ ]:
features_after, _= get_data(test, odd, oddp)

I scaled the features here as well

In [ ]:
scaler=preprocessing.StandardScaler()
col = features_after.columns.tolist()
features_after[col]=scaler.transform(features_after[col])

In [ ]:
prediction=pd.DataFrame(model.predict_proba(features_after.as_matrix()),columns=sorted(categ.unique()))

In [ ]:
prediction.to_csv("abdullah_r1.csv",index_label="Id",na_rep="0")